In [1]:
# insert path containing folders of extracted image
dir_path = '/Users/tinmanee/Downloads/'

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import os
import re
import pandas as pd
from tqdm import tqdm

In [3]:
# Load Model from TF hub
module = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
input_size = 192
def movenet(input_image):
  model = module.signatures['serving_default']

  # SavedModel format expects tensor type of int32.
  input_image = tf.cast(input_image, dtype=tf.int32)
  # Run model inference.
  outputs = model(input_image)
  # Output is a [1, 1, 17, 3] tensor.
  keypoints_with_scores = outputs['output_0'].numpy()
  return keypoints_with_scores

Metal device set to: Apple M1


2023-03-09 16:42:19.823256: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-09 16:42:19.823389: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
# Dictionary that maps from joint names to keypoint indices.
KEYPOINT_DICT = {
    'nose': 0,
    'left_eye': 1,
    'right_eye': 2,
    'left_ear': 3,
    'right_ear': 4,
    'left_shoulder': 5,
    'right_shoulder': 6,
    'left_elbow': 7,
    'right_elbow': 8,
    'left_wrist': 9,
    'right_wrist': 10,
    'left_hip': 11,
    'right_hip': 12,
    'left_knee': 13,
    'right_knee': 14,
    'left_ankle': 15,
    'right_ankle': 16
}

In [5]:
# Get all folders
folders = [entry for entry in os.listdir(dir_path) if os.path.isdir(os.path.join(dir_path, entry))]
folders = [folder for folder in folders if folder.startswith("vid_")]

# find number
vid_nos = []
for i in folders:
    vid_nos.append(int(re.findall(r'\d+', i)[0]))

print(folders)

['vid_75_extracted', 'vid_234_extracted', 'vid_6_extracted', 'vid_72_extracted', 'vid_49_extracted']


In [6]:
# Load the input image.
for i in folders:
    imagefolder_path = os.path.join(dir_path, i)
    data = []
    for frame_no in tqdm(range(len([entry for entry in os.listdir(imagefolder_path) if os.path.isfile(os.path.join(imagefolder_path, entry))]))):
        image_path = imagefolder_path+'/vid_'+re.findall(r'\d+', i)[0]+'frame'+str(frame_no)+".jpg"
        image = tf.io.read_file(image_path)
        image = tf.image.decode_jpeg(image)
        # Resize and pad the image to keep the aspect ratio and fit the expected size.
        input_image = tf.expand_dims(image, axis=0)
        input_image = tf.image.resize_with_pad(input_image, input_size, input_size)

        # Run model inference.
        keypoints_with_scores = movenet(input_image)
        px = pd.DataFrame(keypoints_with_scores[0][0]).astype("float")
        px_flatten = px.to_numpy().flatten()
        data.append(px_flatten)
    df = pd.DataFrame(data, columns=['x0', 'y0', 'c0', 'x1', 'y1', 'c1', 'x2', 'y2', 'c2', 
                                    'x3', 'y3', 'c3', 'x4', 'y4', 'c4', 'x5', 'y5', 'c5', 
                                    'x6', 'y6', 'c6', 'x7', 'y7', 'c7', 'x8', 'y8', 'c8',
                                    'x9', 'y9', 'c9', 'x10', 'y10', 'c10', 'x11', 'y11', 'c11',
                                    'x12', 'y12', 'c12', 'x13', 'y13', 'c13', 'x14', 'y14', 'c14',
                                    'x15', 'y15', 'c15', 'x16', 'y16', 'c16'])
    df.to_csv(imagefolder_path+'/out.csv')

  0%|                                                  | 0/2365 [00:00<?, ?it/s]2023-03-09 16:42:38.128583: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-03-09 16:42:38.136826: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
100%|███████████████████████████████████████| 2898/2898 [02:10<00:00, 22.15it/s]
